# Ex
### Exercise 8.1
- 2, 6, 19, 20, 25

### Exercise 8.2
- 4, 6, 13, 14, 15, 16, 17

### Exercise 8.3
- 3, 6, 7

### Exercise 8.4
- 3, 6, 7, 8, 9

# Com Ex
### Computer Exercise 8.1
- 2, 6, 7

### Computer Exercise 8.2
- 1, 12(a), 12(b)

### Computer Exercise 8.3
- 1(a), 7

### Computer Exercise 8.4
- 3

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
# 8.1 - 2

def dolittle_decomposition(A):
    n = A.shape[0]
    U = A.copy()
    L = np.zeros((n, n))
    
    for i in range(n):
        L[i, i] = 1
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= U[i, i:] * L[j, i]

    return L, U

def crout_decomposition(A):
    n = A.shape[0]
    L, U = dolittle_decomposition(A)
    LT = np.transpose(L)
    D = np.diag(np.diag(U))
    
    return L, D, LT

def verify_dolittle(A):
    L, U = dolittle_decomposition(A)
    
    print("L:\n", L)
    print("U:\n", U)
    print("LU:\n", L @ U)
    print("A:\n", A)
    if np.allclose(L @ U, A):
        print("pass")
    else:
        print("fail")
        
def verify_crout(A):
    L, D, LT = crout_decomposition(A)
    
    print("L:\n", L)
    print("D:\n", D)
    print("LT:\n", LT)
    print("LDT:\n", L @ D @ LT)
    print("A:\n", A)
    if np.allclose(L @ D @ LT, A):
        print("pass")
    else:
        print("fail")

A = np.array(
    [[5, 7, 6, 5], 
     [7, 10, 8, 7], 
     [6, 8, 10, 9], 
     [5, 7, 9, 10]]
    , dtype=float
)

verify_dolittle(A)
print()
verify_crout(A)

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
U:
 [[ 5.   7.   6.   5. ]
 [ 0.   0.2 -0.4  0. ]
 [ 0.   0.   2.   3. ]
 [ 0.   0.   0.   0.5]]
LU:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass

L:
 [[ 1.   0.   0.   0. ]
 [ 1.4  1.   0.   0. ]
 [ 1.2 -2.   1.   0. ]
 [ 1.   0.   1.5  1. ]]
D:
 [[5.  0.  0.  0. ]
 [0.  0.2 0.  0. ]
 [0.  0.  2.  0. ]
 [0.  0.  0.  0.5]]
LT:
 [[ 1.   1.4  1.2  1. ]
 [ 0.   1.  -2.   0. ]
 [ 0.   0.   1.   1.5]
 [ 0.   0.   0.   1. ]]
LDT:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
A:
 [[ 5.  7.  6.  5.]
 [ 7. 10.  8.  7.]
 [ 6.  8. 10.  9.]
 [ 5.  7.  9. 10.]]
pass


In [3]:
# 8.1 - 6, 7

def build_tridiagonal(n, a, b, c):
    A = np.diag([a] * (n - 1), -1) + np.diag([b] * n) + np.diag([c] * (n - 1), 1)
    return A

def inverse_tridiagonal_naive(A):
    L = np.zeros(A.shape)
    U = np.zeros(A.shape) 
    n = A.shape[0]
    
    L[0, 0] = 1
    U[0, :] = A[0, :]
    for i in range(1, n):
        L[i, i] = 1
        L[i, i - 1] = A[i, i - 1] / U[i - 1, i - 1]
        U[i, i] = A[i, i] - U[i - 1, i] * L[i, i - 1]
        if i < n - 1:
            U[i, i + 1] = A[i, i + 1]
        
    L_inv = np.eye(n, dtype=float)
    for i in range(1, n):
        for j in range(0, i):
            L_inv[i, j] = - L[i, i-1] * L_inv[i-1, j]
            
    U_inv = np.zeros((n, n), dtype=float)
    for i in range(n-1, -1, -1):
        U_inv[i, i] = 1.0 / U[i, i]
        for j in range(i+1, n):
            if i + 1 < n:
                U_inv[i, j] = - (U[i, i+1] * U_inv[i+1, j]) / U[i, i]
            else:
                U_inv[i, j] = 0.0
                
    A_inv = U_inv.dot(L_inv)
    return A_inv

def inverse_tridiagonal_thomas(A):
    n = A.shape[0]
    A_inv = np.zeros((n, n), dtype=float)
    I = np.eye(n)
    
    for i in range(n):
        A_inv[:, i] = solve_thomas(A, I[:, i])
    
    return A_inv

def solve_thomas(A, d):
    n = len(d)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    for i in range(n):
        b[i] = A[i][i]
        if i < n - 1:
            c[i] = A[i][i+1]
        if i > 0:
            a[i] = A[i][i-1]
    
    cp = np.zeros(n)
    dp = np.zeros(n)
    
    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]
    
    for i in range(1, n):
        denom = b[i] - a[i] * cp[i - 1]
        if i < n - 1:
            cp[i] = c[i] / denom
        dp[i] = (d[i] - a[i] * dp[i - 1]) / denom
    
    x = np.zeros(n)
    x[-1] = dp[-1]
    for i in range(n - 2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i + 1]
    
    return x
        
def verify_inverse_tridiagonal(A, f):
    A_inv = f(A)
    print("A:\n", A)
    print("A_inv:\n", A_inv)
    print("A_inv @ A:\n", A_inv @ A)
    if np.allclose(A_inv @ A, np.eye(A.shape[0])):
        print("pass")
    else:
        print("fail")
        
verify_inverse_tridiagonal(build_tridiagonal(10, 1, -2, 1), inverse_tridiagonal_thomas)

A:
 [[-2  1  0  0  0  0  0  0  0  0]
 [ 1 -2  1  0  0  0  0  0  0  0]
 [ 0  1 -2  1  0  0  0  0  0  0]
 [ 0  0  1 -2  1  0  0  0  0  0]
 [ 0  0  0  1 -2  1  0  0  0  0]
 [ 0  0  0  0  1 -2  1  0  0  0]
 [ 0  0  0  0  0  1 -2  1  0  0]
 [ 0  0  0  0  0  0  1 -2  1  0]
 [ 0  0  0  0  0  0  0  1 -2  1]
 [ 0  0  0  0  0  0  0  0  1 -2]]
A_inv:
 [[-0.90909091 -0.81818182 -0.72727273 -0.63636364 -0.54545455 -0.45454545
  -0.36363636 -0.27272727 -0.18181818 -0.09090909]
 [-0.81818182 -1.63636364 -1.45454545 -1.27272727 -1.09090909 -0.90909091
  -0.72727273 -0.54545455 -0.36363636 -0.18181818]
 [-0.72727273 -1.45454545 -2.18181818 -1.90909091 -1.63636364 -1.36363636
  -1.09090909 -0.81818182 -0.54545455 -0.27272727]
 [-0.63636364 -1.27272727 -1.90909091 -2.54545455 -2.18181818 -1.81818182
  -1.45454545 -1.09090909 -0.72727273 -0.36363636]
 [-0.54545455 -1.09090909 -1.63636364 -2.18181818 -2.72727273 -2.27272727
  -1.81818182 -1.36363636 -0.90909091 -0.45454545]
 [-0.45454545 -0.90909091 -1.363

In [ ]:
# 8.3 - 1(a)

def rayleigh_quotient(A, x):
    return (x.T @ A @ x) / (x.T @ x)

def power_method(A, max_iter=1000, tol=1e-10):
    x = np.random.rand(A.shape[1])
    
    for _ in range(max_iter):
        Ax = A @ x
        Ax /= np.linalg.norm(Ax)
        if np.linalg.norm(Ax - x) < tol:
            break
        
        x = Ax
        
    return x, rayleigh_quotient(A, x)

def inverse_power_method(A, max_iter=1000, tol=1e-10):
    x = np.random.rand(A.shape[1])
    
    for _ in range(max_iter):
        Ax = np.linalg.solve(A, x)
        Ax /= np.linalg.norm(Ax)
        if np.linalg.norm(Ax - x) < tol:
            break
        
        x = Ax
        
    return x, rayleigh_quotient(A, x)

def power_method_with_shift(A, mu, max_iter=1000, tol=1e-10):
    n = A.shape[0]
    x = np.random.rand(n)
    
    for _ in range(max_iter):   
        Ax = np.linalg.solve(A - mu * np.eye(n), x)
        Ax /= np.linalg.norm(Ax)
        if np.linalg.norm(Ax - x) < tol:
            break
        
        x = Ax
        
    return Ax + mu * x, rayleigh_quotient(A, Ax + mu * x)

def inverse_power_method_with_shift(A, mu, max_iter=1000, tol=1e-10):
    n = A.shape[0]
    x = np.random.rand(n)
    
    for _ in range(max_iter):   
        Ax = np.linalg.solve(A - mu * np.eye(n), x)
        Ax /= np.linalg.norm(Ax)
        if np.linalg.norm(Ax - x) < tol:
            break
        
        x = Ax
        
    return Ax + mu * x, rayleigh_quotient(A, Ax + mu * x)

def aitken_acceleration(x0, x1, x2):
    denom = (x2 - 2*x1 + x0)
    if abs(denom) < 1e-16:
        return x2
    return x0 - (x1 - x0)**2 / denom

def power_method_aitken(A, max_iter=1000, tol=1e-10):
    x = np.random.rand(A.shape[1])
    x /= np.linalg.norm(x)

    lambdas = []

    for k in range(max_iter):
        Ax = A @ x
        Ax /= np.linalg.norm(Ax)

        lam = rayleigh_quotient(A, Ax)
        lambdas.append(lam)

        if k >= 2:
            l0, l1, l2 = lambdas[-3], lambdas[-2], lambdas[-1]
            lam_acc = aitken_acceleration(l0, l1, l2)
            if abs(lam_acc - lambdas[-2]) < tol:
                return Ax, lam_acc

        if np.linalg.norm(Ax - x) < tol:
            break

        x = Ax

    if len(lambdas) >= 3:
        lam_acc = aitken_acceleration(*lambdas[-3:])
    else:
        lam_acc = lambdas[-1]

    return Ax, lam_acc

def inverse_power_method_aitken(A, max_iter=1000, tol=1e-10):
    x = np.random.rand(A.shape[1])
    x /= np.linalg.norm(x)

    lambdas = []

    for k in range(max_iter):
        Ax = np.linalg.solve(A, x)
        Ax /= np.linalg.norm(Ax)

        lam = rayleigh_quotient(A, Ax)
        lambdas.append(lam)

        if k >= 2:
            l0, l1, l2 = lambdas[-3], lambdas[-2], lambdas[-1]
            lam_acc = aitken_acceleration(l0, l1, l2)
            if abs(lam_acc - lambdas[-2]) < tol:
                return Ax, lam_acc

        if np.linalg.norm(Ax - x) < tol:
            break

        x = Ax

    if len(lambdas) >= 3:
        lam_acc = aitken_acceleration(*lambdas[-3:])
    else:
        lam_acc = lambdas[-1]

    return Ax, lam_acc

A = np.array(
    [[5, 4, 1, 1], 
     [4, 5, 1, 1],
     [1, 1, 4, 2], 
     [1, 1, 2, 4]]
)

eigval, eigvec = np.linalg.eig(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method")
eigvec, eigval = power_method(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method")
eigvec, eigval = inverse_power_method(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method with Shift")
eigvec, eigval = power_method_with_shift(A, 4.9)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method with Shift")
eigvec, eigval = inverse_power_method_with_shift(A, 4.9)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method with Aitken Acceleration")
eigvec, eigval = power_method_aitken(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method with Aitken Acceleration")
eigvec, eigval = inverse_power_method_aitken(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)

Eigenvalue: [10.  1.  5.  2.]
Eigenvector: [[-6.32455532e-01 -7.07106781e-01  3.16227766e-01  1.66533454e-16]
 [-6.32455532e-01  7.07106781e-01  3.16227766e-01 -6.74454005e-17]
 [-3.16227766e-01 -5.94276718e-17 -6.32455532e-01 -7.07106781e-01]
 [-3.16227766e-01 -8.70027738e-17 -6.32455532e-01  7.07106781e-01]]

Power Method
Eigenvalue: 10.0
Eigenvector: [0.63245553 0.63245553 0.31622777 0.31622777]

Inverse Power Method
Eigenvalue: 0.9999999999999998
Eigenvector: [-7.07106781e-01  7.07106781e-01 -1.02149566e-10  1.02149581e-10]

Power Method with Shift
Eigenvalue: 5.0
Eigenvector: [-1.86574382 -1.86574382  3.73148764  3.73148764]

Inverse Power Method with Shift
Eigenvalue: 5.0
Eigenvector: [-1.86574382 -1.86574382  3.73148764  3.73148764]

Power Method with Aitken Acceleration
Eigenvalue: 10.000000000000004
Eigenvector: [0.63245496 0.63245496 0.3162289  0.3162289 ]

Inverse Power Method with Aitken Acceleration
Eigenvalue: 1.0
Eigenvector: [ 7.07106781e-01 -7.07106781e-01 -2.01317389e

In [16]:
# 8.3 - 7

A = np.array(
    [[-4, 14, 0],
     [-5, 13, 0],
     [-1, 0, 2]]
)

eigval, eigvec = np.linalg.eig(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method")
eigvec, eigval = power_method(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method")
eigvec, eigval = inverse_power_method(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method with Shift")
eigvec, eigval = power_method_with_shift(A, 4.9)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method with Shift")
eigvec, eigval = inverse_power_method_with_shift(A, 4.9)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Power Method with Aitken Acceleration")
eigvec, eigval = power_method_aitken(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)
print()
print("Inverse Power Method with Aitken Acceleration")
eigvec, eigval = inverse_power_method_aitken(A)
print("Eigenvalue:", eigval)
print("Eigenvector:", eigvec)

Eigenvalue: [2. 6. 3.]
Eigenvector: [[ 0.          0.79740048  0.66666667]
 [ 0.          0.56957177  0.33333333]
 [ 1.         -0.19935012 -0.66666667]]

Power Method
Eigenvalue: 6.000000000602557
Eigenvector: [-0.79740048 -0.56957177  0.19935012]

Inverse Power Method
Eigenvalue: 1.9999999997681484
Eigenvector: [-2.31851592e-10 -1.15925796e-10 -1.00000000e+00]

Power Method with Shift
Eigenvalue: 6.000000000201389
Eigenvector: [-4.70466284 -3.36047345  1.17616571]

Inverse Power Method with Shift
Eigenvalue: 6.0000000001633085
Eigenvector: [ 4.70466284  3.36047345 -1.17616571]

Power Method with Aitken Acceleration
Eigenvalue: 6.000000000000001
Eigenvector: [ 0.79740048  0.56957177 -0.19935012]

Inverse Power Method with Aitken Acceleration
Eigenvalue: 2.000000000000002
Eigenvector: [1.74179423e-10 8.70897113e-11 1.00000000e+00]


In [ ]:
# 8.4 - 3

# jacobi iteration using diagonal elements of A
# if A is not diagonally dominant, it may not converge
def jacobi_iteration(A, b, x0=None, max_iter=1000, tol=1e-4):
    n = A.shape[0]
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0.copy()

    D = np.diag(A)                       # 대각성분 벡터
    R = A - np.diagflat(D)               # 나머지 성분

    iter = 0
    for _ in range(max_iter):
        iter += 1
        x_new = (b - R @ x) / D          # Jacobi 업데이트
        if np.linalg.norm(x_new - x) < tol:
            return x_new, iter
        x = x_new

    return x, iter

def gauss_seidel_iteration(A, b, x0=None, max_iter=1000, tol=1e-4):
    n = A.shape[0]
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0.copy()

    iter = 0
    for _ in range(max_iter):
        iter += 1
        x_old = x.copy()

        for i in range(n):
            sum1 = A[i, :i]   @ x[:i]
            sum2 = A[i, i+1:] @ x_old[i+1:]
            x[i] = (b[i] - sum1 - sum2) / A[i, i]

        if np.linalg.norm(x - x_old) < tol:
            return x, iter

    return x, iter

def sor_iteration(A, b, w=1.2, x0=None, max_iter=1000, tol=1e-4):
    n = A.shape[0]
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0.copy()

    iter = 0
    for _ in range(max_iter):
        iter += 1   
        x_old = x.copy()

        for i in range(n):
            sum1 = A[i, :i]   @ x[:i]
            sum2 = A[i, i+1:] @ x_old[i+1:]
            x[i] = (1 - w) * x_old[i] + (w / A[i, i]) * (b[i] - sum1 - sum2)

        if np.linalg.norm(x - x_old) < tol:
            return x, iter

    return x, iter
        
def verify_solver(A, b, f, x0=None):
    print("Solver:", f.__name__)
    x, iter = f(A, b, x0=x0)
    print("x:\n", x)
    print("Ax:\n", A @ x)
    print("b:\n", b)
    print("Iterations:", iter)
    print("Residual:", np.linalg.norm(A @ x - b))
        
A = np.array(
    [[7, 3, -1, 2],
     [3, 8, 1, -4],
     [-1, 1, 4, -1],
     [2, -4, -1, 6]]
)

b = np.array([-1, 0, -3, 1])

verify_solver(A, b, jacobi_iteration)
print()
verify_solver(A, b, gauss_seidel_iteration)
print()
verify_solver(A, b, sor_iteration)

Solver: jacobi_iteration


ValueError: too many values to unpack (expected 2)